In [1]:
import csv
restaurants_list = list()
restaurants_title = list()
restaurants_address = list()
f = open("C:/Users/SSAFY/Desktop/2학기/02_특화프로젝트/02_데이터/busan_list.csv")
csvfile = csv.reader(f)
for title,address in csvfile :
    restaurants_list.append(title+' '+address)
    restaurants_title.append(title)
    restaurants_address.append(address)
print(restaurants_list)

['조방낙지 부산광역시 강서구 대저1동 333-7', '서민삼겹 부산광역시 연제구 연산동 729-8', '경대컵밥 만덕점 부산광역시 북구 만덕동 849', '하대승담양갈비 가야점 부산광역시 부산진구 가야동 56-2', '서울깍두기 부산광역시 부산진구 부전동 125-6', '울릉도소주방 부산광역시 부산진구 양정동 393-10', '대복집 부산광역시 연제구 연산동 1514-1', '오케이회포차 부산광역시 북구 만덕동 311', '유메노다이닝 부산광역시 금정구 구서동 446-18', '양반게장 전포직영점 부산광역시 부산진구 전포동 558', '양빠 부산광역시 중구 동광동1가 8-3', '대학식당 부산광역시 서구 아미동2가 9-10', '가마치통닭 명장점 부산광역시 동래구 명장동 326-4', '루흐 부산광역시 중구 광복동1가 56-1', '가야한방밀면 부산광역시 부산진구 부전동 265-6', '본가가야밀면 부산광역시 남구 대연동 513-7', '역전할머니맥주 부산서동점 부산광역시 금정구 서동 205-1', '족장의칼 안락점 부산광역시 동래구 안락동 603-8', '가미 부산광역시 해운대구 우동 1520 롯데갤러리움', '모던테이블 부산광역시 부산진구 부전동 220-1', '빛날빈 부산광역시 중구 광복동2가 2-86 1, 2층', '어반테이블 부산광역시 강서구 명지동 3260-4', '홈팝참치 서동점 부산광역시 금정구 서동 204-13', '문현곱창 부산광역시 사하구 장림동 307', '옆집아줌마 부산광역시 동래구 안락동 243-1', '쪽닭 부산광역시 북구 만덕동 835-38', '영호회집 부산광역시 서구 충무동1가 14-30', '오륙도낙지 부산광역시 동구 범일동 181-1', '황궁쟁반짜장 부산광역시 해운대구 반송동 40-573', '원향루 부산광역시 사하구 하단동 510-1', '또순이실비 부산광역시 사하구 괴정동 954-5', '오케이조개구이 부산광역시 동래구 온천동 707-1', '맘스터치 부산대청점 부산광역시 중구 대청동2가 22-6', '하삼동커피 자갈

In [19]:
!pip install selenium

In [23]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import time
from bs4 import BeautifulSoup
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("--incognito")
# chrome_options.add_argument("--headless")
# chrome_options.add_argument("--no-sandbox")
# chrome_options.add_argument("--disable-setuid-sandbox")
# chrome_options.add_argument('--disable-dev-shm-usage')
# chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

In [8]:
print(len(restaurants_list))
print(len(restaurants_title))
print(restaurants_list[0])

12143
12143
조방낙지 부산광역시 강서구 대저1동 333-7


In [4]:
columns = ['title','subway','reviewer','review','rank']
df_restaurants = pd.DataFrame(columns=columns)

columns2 = ['title','address']
df_unsearched = pd.DataFrame(columns=columns2)


In [6]:
# 검색

# restaurants = ['조방낙지 부산광역시 강서구 대저1동 333-7', '서민삼겹 부산광역시 연제구 연산동 729-8','보배반점 해운대좌동점 부산광역시 해운대구 좌동 862', '구디너프 부산광역시 금정구 장전동 299-47']
for i in range(3659,7207) :
    
    driver = webdriver.Chrome('C:/Users/SSAFY/Desktop/selenium/chromedriver.exe',chrome_options=chrome_options)
    driver.implicitly_wait(5)
    url = 'https://map.naver.com/v5/search'
    driver.get(url)
    driver.implicitly_wait(5)
    search_box = driver.find_element_by_css_selector('div.input_box>input.input_search')
    driver.implicitly_wait(5)
    search_box.send_keys(restaurants_list[i])
    driver.implicitly_wait(5)
    search_box.send_keys(Keys.ENTER)
    driver.implicitly_wait(5)
    
    try :
        searchiframe = driver.find_element_by_id("searchIframe")
        driver.switch_to.frame(searchiframe)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser') 

        driver.switch_to.default_content()

        entryIframe = driver.find_element_by_id("entryIframe")
        driver.switch_to.frame(entryIframe)

        html = driver.page_source # 클릭 후 페이지가져오기
        soup = BeautifulSoup(html, 'html.parser')
        place_detail_wrapper = soup.select_one('div.place_detail_wrapper')
    except:
        df_unsearched = df_unsearched.append({
            'title': restaurants_title[i],
            'address' : restaurants_address[i]
        },ignore_index=True)
        print('조회 결과 없음 :'+restaurants_title[i])

    try :


        # 가게이름
        try :
            title = place_detail_wrapper.select('span._3XamX')[0].text
            print(title)
        except :
            title = ''
        
            
        # 지하철
        try :
            subway_tag = place_detail_wrapper.select('div._2P6sT')[0]
            while 1 :
                if subway_tag.find('span') :
                    subway_tag.find('span').decompose() # span태그 빼기
                else :
                    break
            subway = subway_tag.text
            print(subway)
        except :
            subway =''
        
        # 리뷰
        driver.implicitly_wait(5)
        find_review_table = soup.select_one('div._2MDmw')
        driver.implicitly_wait(5)
        # print(driver.find_element_by_css_selector('div.place_detail_wrapper > div.place_fixed_maintab > div > div > div > div > a:nth-child(1) > span')
        for i in range(len(find_review_table)) :
            path = 'div.place_detail_wrapper > div.place_fixed_maintab > div > div > div > div > a:nth-child({0}) > span'.format(i+1)
            if driver.find_element_by_css_selector(path).text == '리뷰':
                driver.find_element_by_css_selector(path).click()
                break

        flag = 1
        click_n = 0
        driver.implicitly_wait(5)
        while flag :
            if click_n >= 10 :
                break
            try :
                driver.find_element_by_css_selector('div._2kAri').click()
                time.sleep(0.5)
                click_n += 1
            except :
                flag = 0
        html = driver.page_source # 내리고 나서의 페이지를 싹 가져옴
        soup = BeautifulSoup(html, 'html.parser') 
        reviews_ul = soup.select_one('ul._1jVSG')
        reviews_li = reviews_ul.select('li')

        for i in range(len(reviews_li)):
#             print(len(reviews_li))
            
            #리뷰어
            try :
                reviewer = reviews_li[i].select('div._16RxQ')[0].text
#                 print(reviewer)
            except :
                reviewer = ''
            #리뷰
            try :
                review = reviews_li[i].select('span.WoYOw')[0].text
#                 print(review)
            except :
                review = ''
            # 랭크
            try :
                rank = reviews_li[i].select('span.Sv1wj')[0].text[2:]
#                 print(rank)
            except :
                rank = ''
#                 print(rank)
            df_restaurants = df_restaurants.append({
                'title':title,
                'subway':subway,
                'reviewer' : reviewer,
                'review':review,
                'rank':rank
            },ignore_index=True)
            
    except :
        pass

    driver.switch_to.default_content()
    print(len(df_restaurants))
    
    driver.quit()
#     time.sleep(1)


남산골오리와삼겹
남산정역 4번 출구에서422m
0
0
조회 결과 없음 :남해집
0
조회 결과 없음 :죽이야기 부산전포점
0
카페봉봉
수안역 6번 출구에서222m
1
1
착한소
서동역 1번 출구에서336m
9
조회 결과 없음 :코코플라토
착한소
서동역 1번 출구에서336m
9
9
김밥이야기 개금본점
개금역 2번 출구에서680m
38
조회 결과 없음 :이디야
김밥이야기 개금본점
개금역 2번 출구에서680m
38
다찌
41
호맥
남포역 7번 출구에서180m
93
조회 결과 없음 :원웨이
호맥
남포역 7번 출구에서180m
93
멜트
명륜역 5번 출구에서527m
102
리멘시타
부산대역 3번 출구에서491m
140
조회 결과 없음 :자매분식
리멘시타
부산대역 3번 출구에서491m
140
태영아구찜
장전역 3번 출구에서565m
145
조방낙지사상직영점
괘법 르네시떼역 1번 출구에서368m
224
술나방바
서면역 6번 출구에서274m
316
영커피 서면점
서면역 2번 출구에서437m
328
조회 결과 없음 :라멘만땅
영커피 서면점
서면역 2번 출구에서437m
328
리틀프랑 본점
부산대역 3번 출구에서371m
425
부산돼지국밥
남천역 1번 출구에서459m
508
맛나손칼국수
568
568
문버거
678
롯데리아 부산대연점
대연역 5번 출구에서288m
768
조회 결과 없음 :청와대숯불갈비
롯데리아 부산대연점
대연역 5번 출구에서288m
768
베이징
동의대역 5번 출구에서741m
774
버거킹 부산태화점
서면역 2번 출구에서260m
884
884
도농한우전문시식당
남천역 1번 출구에서173m
894
조회 결과 없음 :제일분식
도농한우전문시식당
남천역 1번 출구에서173m
894
몽블랑
자갈치역 7번 출구에서442m
897
홍돈
하단역 11번 출구에서136m
930
조회 결과 없음 :본죽 대연점
홍돈
하단역 11번 출구에서136m
930
목화기사식당
명륜역 3번 출구에서485m
1040
조회 결과 없음 :카이루이자까야
목화기사식당
명륜역 3번 출구에서485m
1040
대가

황궁쟁반짜장
사하역 1번 출구에서402m
8203
오가다 부산 동의대점
동의대역 3번 출구에서746m
8229
통낙지
국제금융센터.부산은행역 3번 출구에서22m
8238
영도횟집
남포역 2번 출구에서278m
8240
한우뭉치 해운대점
장산역 4번 출구에서496m
8252
낚시고기횟집
시청역 2번 출구에서160m
8275
카페태그
배산역 1번 출구에서420m
8276
카페다대포여기
다대포항역 4번 출구에서258m
8333
8333
조회 결과 없음 :역전할머니맥주 부산동대신점
8333
조회 결과 없음 :신보성
8333
조회 결과 없음 :아방궁
8333
포앤스마일 락사
경성대.부경대역 1번 출구에서284m
8361
바오반
경성대.부경대역 1번 출구에서283m
8417
조회 결과 없음 :롤링파스타 명지점
바오반
경성대.부경대역 1번 출구에서283m
8417
우돼끼리 서면직영점
서면역 2번 출구에서72m
8428
진성원
하단역 11번 출구에서222m
8436
8436
조회 결과 없음 :광안리남매
8436
조회 결과 없음 :주례칼국수
8436
조회 결과 없음 :스타벅스 부전역점
8436
조회 결과 없음 :볼로냐
8436
조회 결과 없음 :화전국수
8436
조회 결과 없음 :신토불이
8436
조회 결과 없음 :리리탐뷔페
8436
조회 결과 없음 :라로커피
8436
막썰어횟집
광안역 2번 출구에서149m
8457
아지트
토성역 8번 출구에서6m
8457
순곱이네 장림점
신장림역 1번 출구에서857m
8458
양정정통밀면
양정역 1번 출구에서20m
8568
조회 결과 없음 :대박해물탕해물찜
양정정통밀면
양정역 1번 출구에서20m
8568
신쭈꾸미 사직점
사직역 1번 출구에서753m
8602
오후의홍차
민락역 1번 출구에서594m
8712
고반식당 서면점
서면역 7번 출구에서321m
8812
조회 결과 없음 :블루샥 부산괴정점
고반식당 서면점
서면역 7번 출구에서321m
8812
조회 결과 없음 :정가네샤브샤브
고반식당 서면점
서면역 7번 출구에서321m
8812
조회 결과 없

오사카삼겹살괴정점
괴정역 2번 출구에서15m
15881
조회 결과 없음 :시간
오사카삼겹살괴정점
괴정역 2번 출구에서15m
15881
호맥
전포역 7번 출구에서299m
15991
서요 재송점
센텀역 1번 출구에서274m
16010
삼성반점
주례역 7번 출구에서948m
16012
16012
배가네 마약쭈꾸미
온천장역 5번 출구에서606m
16014
고기굽는남자
서면역 6번 출구에서231m
16124
야끼니꾸 다깡
부산대역 3번 출구에서435m
16140
16140
16140
조회 결과 없음 :5FLOOR
16140
조회 결과 없음 :부산밀면
16140
두메산골돼지국밥
모라역 1번 출구에서465m
16141
16141
가배원커피
센텀시티역 4번 출구에서636m
16182
식감
못골역 1번 출구에서277m
16183
도야족발 화명점
화명역 1번 출구에서492m
16223
컴포즈커피 부산교대점
교대역 3번 출구에서144m
16274
16274
조회 결과 없음 :마당쇠 빨간쭈꾸미
16274
식가당
하단역 7번 출구에서1064m
16315
79대포 경성대부경대점
경성대.부경대역 1번 출구에서135m
16325
전주남문토종순대국 전포점
부전역 6번 출구에서100m
16390
소소한한우
낫개역 4번 출구에서608m
16403
16403
16403
신차이나
부전역 1번 출구에서203m
16409
김가네빈대떡
장산역 10번 출구에서206m
16434
진주네
덕포역 4번 출구에서513m
16442
삼진어묵 신세계백화점 센텀시티점
센텀시티역 12번 출구에서63m
16552
당그레양곱창구이
해운대역 4번 출구에서405m
16582
사직곱창
사직역 3번 출구에서690m
16587
16587
16587
싱싱해싱싱어수산
덕천역 9번 출구에서346m
16617
부엉이식당 남천점
남천역 3번 출구에서284m
16647
16647
하단밀면전문점
하단역 7번 출구에서1174m
16667
조회 결과 없음 :임광선 커피하우스
하단밀면전문점
하단역 7번 출구에서1174m
16667
16667
조회 결

23595
돈모리
두실역 7번 출구에서633m
23641
런치박스
센텀시티역 3번 출구에서233m
23649
삼정식당
경성대.부경대역 5번 출구에서476m
23658
고금국밥
거제역 8번 출구에서188m
23669
홍차왕자
서대신역 4번 출구에서237m
23670
조회 결과 없음 :진성횟집
홍차왕자
서대신역 4번 출구에서237m
23670
쿨키즈네버다이
장전역 1번 출구에서312m
23708
맛있는 즉석김밥
대연역 1번 출구에서768m
23739
빨강포차
부암역 1번 출구에서574m
23742
조회 결과 없음 :용이네뒷고기
빨강포차
부암역 1번 출구에서574m
23742
23742
언양숯불갈비
23852
제주이마이가남산점
남산역 7번 출구에서478m
23877
조회 결과 없음 :친정엄마 아라수라간
제주이마이가남산점
남산역 7번 출구에서478m
23877
텐퍼센트커피 연산연일점
연산역 8번 출구에서127m
23895
그린샤브앤쭈꾸미 본점
23933
조회 결과 없음 :엔그램 커피 로스터리
그린샤브앤쭈꾸미 본점
23933
조회 결과 없음 :옥싸롱
그린샤브앤쭈꾸미 본점
23933
23933
23933
혼돈
23942
조회 결과 없음 :기장멸치국수
혼돈
23942
그리고칠월
동대신역 8번 출구에서396m
23954
홍대앞맥주집 양정점
양정역 1번 출구에서204m
23976
조회 결과 없음 :민속촌물레방아설렁탕
홍대앞맥주집 양정점
양정역 1번 출구에서204m
23976
도쿠도쿠야
서면역 6번 출구에서266m
23989
온리더치
사상역 6번 출구에서308m
24088
조회 결과 없음 :섬데이
온리더치
사상역 6번 출구에서308m
24088
광복밀면
좌천역 4번 출구에서482m
24107
통발횟집
수안역 7번 출구에서291m
24108
봄베이브로이 광안점
금련산역 1번 출구에서615m
24116
하삼동커피 금곡점
금곡역 1번 출구에서548m
24164
데일리팡
개금역 1번 출구에서369m
24274
조회 결과 없음 :지리산어탕국수
데일리팡
개금역 1번 출구에서369m
242

30587
호호사직닭발 양정점
양정역 1번 출구에서307m
30593
30593
전주남문토종순대국 센텀점
센텀역 2번 출구에서623m
30663
조회 결과 없음 :밀화 신세계백화점 센텀시티점
전주남문토종순대국 센텀점
센텀역 2번 출구에서623m
30663
찌짐과막걸리집
30670
조회 결과 없음 :소풍경 서구구덕운동장점
찌짐과막걸리집
30670
홍복작은면가게
30747
30747
가온비
자갈치역 1번 출구에서304m
30765
조회 결과 없음 :남해수산횟집
가온비
자갈치역 1번 출구에서304m
30765
조회 결과 없음 :엔제리너스 부산아이온시티점
가온비
자갈치역 1번 출구에서304m
30765
킹프레소 다대점
다대포항역 1번 출구에서29m
30788
조회 결과 없음 :유가네닭갈비 부산시청점
킹프레소 다대점
다대포항역 1번 출구에서29m
30788
30788
30788
30788
조회 결과 없음 :해돋이 아구동태찜탕
30788
자수정코다리
대연역 6번 출구에서285m
30852
조회 결과 없음 :투다리
자수정코다리
대연역 6번 출구에서285m
30852
육회마을 덕천점
덕천역 9번 출구에서160m
30868
30868
나가하마만게츠
해운대역 4번 출구에서408m
30968
달토끼포차
전포역 7번 출구에서285m
30973
30973
인생닭강정 부산만덕점
만덕역 3번 출구에서667m
30983
이플다이닝
부산대역 1번 출구에서618m
30989
30989
조회 결과 없음 :전주콩나물국밥
30989
30989
30989
신전떡볶이 남천점
남천역 1번 출구에서93m
31031
조회 결과 없음 :가마솥등뼈감자탕
신전떡볶이 남천점
남천역 1번 출구에서93m
31031
김가네김밥 엄궁점
31115
빨간병아리 서면본점
서면역 1번 출구에서418m
31140
조회 결과 없음 :이웃집
빨간병아리 서면본점
서면역 1번 출구에서418m
31140
부산해물탕해물찜
화명역 1번 출구에서192m
31215
조회 결과 없음 :최고당돈가스
부산해물탕해물찜
화명역 1번 출구에서192m
31

...

60793
기장손칼국수
서면역 7번 출구에서181m
60903
60903
60903
버거킹 부산명지DT점
61013
조회 결과 없음 :블루문
버거킹 부산명지DT점
61013
조회 결과 없음 :서울깍두기
버거킹 부산명지DT점
61013
취성루
수정역 2번 출구에서27m
61014
동원양곱창12호
자갈치역 6번 출구에서118m
61018
해장국집
동대신역 6번 출구에서202m
61019
조회 결과 없음 :주식회사
해장국집
동대신역 6번 출구에서202m
61019
조회 결과 없음 :황재성 열라짬뽕 부산거제법원점
해장국집
동대신역 6번 출구에서202m
61019
조회 결과 없음 :J.Brews
해장국집
동대신역 6번 출구에서202m
61019
조회 결과 없음 :수라비빔국수
해장국집
동대신역 6번 출구에서202m
61019
쇼코디저트
전포역 8번 출구에서115m
61028
금샘다방
범어사역 1번 출구에서830m
61138
조회 결과 없음 :신흥반점
금샘다방
범어사역 1번 출구에서830m
61138
긴타로 하단점
하단역 3번 출구에서171m
61170
61170
61170
카페본동
개금역 2번 출구에서761m
61280
헬로크랩 화명점
화명역 1번 출구에서297m
61313
조회 결과 없음 :플랜티모어 부산시립미술관내
헬로크랩 화명점
화명역 1번 출구에서297m
61313
61313
밥상예찬
덕천역 5번 출구에서180m
61409
조회 결과 없음 :후니스시
밥상예찬
덕천역 5번 출구에서180m
61409
고봉민김밥인 부산수안점
수안역 4번 출구에서121m
61504
유로실비
전포역 1번 출구에서300m
61522
띵크커피 부산센텀점
센텀시티역 6번 출구에서539m
61603
조회 결과 없음 :보타닉 아덴
띵크커피 부산센텀점
센텀시티역 6번 출구에서539m
61603
조회 결과 없음 :두꺼비
띵크커피 부산센텀점
센텀시티역 6번 출구에서539m
61603
백년족발
남산역 7번 출구에서303m
61606
이디야 연산대로점
연산역 5번 출구에서266m
61642
조회 결과 없음

68605
68605
68605
조회 결과 없음 :고려홍삼삼계탕
68605
악당떡볶이
거제해맞이역 2번 출구에서389m
68613
2Chef 본점
중앙역 7번 출구에서82m
68657
68657
E&O 신세계백화점센텀시티점
센텀시티역 12번 출구에서63m
68683
대양상회
남포역 2번 출구에서278m
68749
조회 결과 없음 :키친번트
대양상회
남포역 2번 출구에서278m
68749
수제돈까스토마토
장산역 5번 출구에서54m
68859
68859
궁채
덕천역 12번 출구에서98m
68879
68879
남궁
68910
투다리 지사점
68913
수안커피컴퍼니 플래그십 스토어
수안역 8번 출구에서356m
69023
69023
생활맥주 부산서면점
서면역 6번 출구에서229m
69060
조회 결과 없음 :오늘은생선구이
생활맥주 부산서면점
서면역 6번 출구에서229m
69060
일광양곱창
자갈치역 10번 출구에서75m
69094
조회 결과 없음 :23년돼지국밥
일광양곱창
자갈치역 10번 출구에서75m
69094
노리터BAR
범일역 2번 출구에서513m
69095
소문난주문진막국수
부산진역 7번 출구에서189m
69096
마리스커피
자갈치역 1번 출구에서152m
69100
69100
베르데
69166
롯데리아 하나로부산점
율리역 3번 출구에서409m
69276
조회 결과 없음 :이삭토스트
롯데리아 하나로부산점
율리역 3번 출구에서409m
69276
조회 결과 없음 :바리스트로 명륜아이파크점
롯데리아 하나로부산점
율리역 3번 출구에서409m
69276
손영환비빔국수
수영역 1번 출구에서31m
69307
69307
조회 결과 없음 :정성식당
69307
이연
전포역 8번 출구에서356m
69316
조회 결과 없음 :소머리국밥
이연
전포역 8번 출구에서356m
69316
도야족발 동래점
명륜역 5번 출구에서592m
69346
조회 결과 없음 :콕스테일
도야족발 동래점
명륜역 5번 출구에서592m
69346
69346
시골집
서면역 13번 출구에서316m
69361
조회 결과 

원뿔한우참숯구이
장산역 4번 출구에서431m
75668
광안리 끄티집
광안역 1번 출구에서882m
75778
75778
동백아가씨1961
서면역 8번 출구에서438m
75888
조회 결과 없음 :댄싱컴 범일점
동백아가씨1961
서면역 8번 출구에서438m
75888
킹콩부대찌개 부산 서면롯데점
서면역 7번 출구에서298m
75948
육화정
동대신역 2번 출구에서413m
75975
75975
서면 고메
전포역 5번 출구에서63m
76085
조회 결과 없음 :남산중국관
서면 고메
전포역 5번 출구에서63m
76085
프띠르
전포역 7번 출구에서372m
76195
나의아저씨
온천장역 1번 출구에서618m
76197
조회 결과 없음 :대동국수
나의아저씨
온천장역 1번 출구에서618m
76197
조회 결과 없음 :셀렉토커피 부산괴정점
나의아저씨
온천장역 1번 출구에서618m
76197
이츠키
부산대역 3번 출구에서447m
76207
조회 결과 없음 :리미니
이츠키
부산대역 3번 출구에서447m
76207
이디야커피 부산범어사역점
범어사역 6번 출구에서4m
76247
투썸플레이스 부산대한통운점
중앙역 2번 출구에서235m
76277
이조숯불갈비 이조정식
서대신역 4번 출구에서327m
76299
진국명가 온천천점
동래역 2번 출구에서668m
76409
조회 결과 없음 :외가집
진국명가 온천천점
동래역 2번 출구에서668m
76409
성일손칼국수
자갈치역 2번 출구에서150m
76416
준짬뽕
부암역 1번 출구에서574m
76472
76472
길 세이로무시 동래점
수안역 3번 출구에서306m
76572
조회 결과 없음 :에그박스 부산하단점
길 세이로무시 동래점
수안역 3번 출구에서306m
76572
조회 결과 없음 :이차돌 부산서면점
길 세이로무시 동래점
수안역 3번 출구에서306m
76572
수영동우담
수영역 5번 출구에서232m
76572
76572
웨이아웃
토성역 2번 출구에서36m
76588
금용
76619
조회 결과 없음 :미니펍
금용
76619
조회 결과 없음 :

81917
유가네닭갈비 수영점
수영역 2번 출구에서42m
82008
빈체스트
교대역 1번 출구에서65m
82091
카페토드
동매역 2번 출구에서198m
82158
철근소금구이
서동역 3번 출구에서190m
82162
조회 결과 없음 :상황삼계탕
철근소금구이
서동역 3번 출구에서190m
82162
조회 결과 없음 :골목집
철근소금구이
서동역 3번 출구에서190m
82162
감천집밥뷔페
82175
대풍각
동의대역 3번 출구에서697m
82211
무지개반점
주례역 1번 출구에서923m
82229
미진축산 부산대점
부산대역 3번 출구에서426m
82239
조회 결과 없음 :쌍용이포차
미진축산 부산대점
부산대역 3번 출구에서426m
82239
소문난초량할매쭈꾸미 서원시장직영점
충렬사역 4번 출구에서309m
82255
대박식당
중앙역 5번 출구에서543m
82262
박장군삼겹살
다대포항역 1번 출구에서387m
82263
일송횟집
충렬사역 2번 출구에서266m
82273
조회 결과 없음 :다복국수
일송횟집
충렬사역 2번 출구에서266m
82273
가마치통닭 화명롯데점
화명역 3번 출구에서293m
82374
오별난멸치국수 범일점
범일역 2번 출구에서211m
82474
재성밀면 감천직영점
82535
블랙업커피 서면
서면역 2번 출구에서146m
82645
82645
경양카츠X경양선술 동래점
동래역 2번 출구에서186m
82755
파스타웨이브
토성역 10번 출구에서90m
82755
서울깍두기
82756
도모2012
서면역 1번 출구에서559m
82757
스타시티누벨바그
낫개역 4번 출구에서608m
82757
생활맥주 광안리점
금련산역 1번 출구에서537m
82866
82866
베러먼데이 장산파밀리에점
장산역 14번 출구에서27m
82932
조회 결과 없음 :자갈치산꼼장어
베러먼데이 장산파밀리에점
장산역 14번 출구에서27m
82932
바다횟집
남포역 2번 출구에서236m
83042
83042
인디고
경성대.부경대역 1번 출구에서440m
83047
자갈치브라더스 사하구평점
장림역 4

KeyboardInterrupt: 

In [157]:
df_restaurants= df_restaurants.drop(labels=range(0,3048))
print(df_restaurants)

Empty DataFrame
Columns: [title, subway, reviewer, review, rank]
Index: []


In [183]:
# df_unsearched = df_unsearched.drop(labels=range(0,25))
print(df_unsearched)

        title                        address
0       서울깍두기           부산광역시 부산진구 부전동 125-6
1         대복집           부산광역시 연제구 연산동 1514-1
2        대학식당            부산광역시 서구 아미동2가 9-10
3          루흐            부산광역시 중구 광복동1가 56-1
4      본가가야밀면             부산광역시 남구 대연동 513-7
..        ...                            ...
680       복성루            부산광역시 사상구 괘법동 379-8
681       린마벨  부산광역시 사하구 괴정동 1284 302동 b206호
682    아촌숯불갈비           부산광역시 금정구 부곡동 896-27
683      다대횟집           부산광역시 사하구 하단동 1165-9
684  카셀라 1978             부산광역시 금정구 부곡동 63-1

[685 rows x 2 columns]


In [7]:
df_restaurants.to_csv(path_or_buf='음식점리뷰_ver3.csv',encoding='utf-8-sig')
df_unsearched.to_csv(path_or_buf='음식점리뷰X_ver3.csv',encoding='utf-8-sig')

In [ ]:
# 크롤링 안된 애들 마저 크롤링하기

In [18]:
import csv
restaurants_list = list()
restaurants_title = list()
restaurants_address = list()
f = open("./음식점리뷰/unsearched_merge1_crawling_ver.csv")
csvfile = csv.reader(f)
csvfile
for title,address in csvfile :
    address_ls = []
    cnt = 0
    for word in address.split(" ") :
        if cnt > 2 :
            break
        else :
            address_ls.append(word)
            cnt += 1

    restaurants_list.append(title+' '+' '.join(address_ls))
    restaurants_title.append(title)
    restaurants_address.append(address)
print(restaurants_list)

['서울깍두기 부산광역시 부산진구 부전동', '대복집 부산광역시 연제구 연산동', '대학식당 부산광역시 서구 아미동2가', '루흐 부산광역시 중구 광복동1가', '본가가야밀면 부산광역시 남구 대연동', '가미 부산광역시 해운대구 우동', '빛날빈 부산광역시 중구 광복동2가', '문현곱창 부산광역시 사하구 장림동', '오륙도낙지 부산광역시 동구 범일동', '원향루 부산광역시 사하구 하단동', '또순이실비 부산광역시 사하구 괴정동', '오케이조개구이 부산광역시 동래구 온천동', '맘스터치 부산대청점 부산광역시 중구 대청동2가', '스시투어 경성대점 부산광역시 남구 대연동', '뜨락쌈전문점 부산광역시 부산진구 부전동', '인생닭강정 온천장역점 부산광역시 금정구 부곡동', '몰타 부산광역시 금정구 장전동', '덕천복집 부산광역시 북구 덕천동', '산호장횟집앤조개구이 부산광역시 사하구 다대동', '오늘김해뒷고기 부산광역시 남구 대연동', '홍콩반점0410 서면점 부산광역시 부산진구 부전동', '면앤면추억의가락국수 부산광역시 동구 초량동', '밀양국밥 부산광역시 사상구 괘법동', '파티넥스트도어 부산광역시 부산진구 부전동', '월강 부산광역시 부산진구 부전동', '초가집양곱창 부산광역시 서구 동대신동2가', '강남할매집 부산광역시 동래구 온천동', '너랑나랑분식 부산광역시 사하구 하단동', '연산숯불왕갈비 부산광역시 해운대구 우동', '엔제리너스 부산광역시 부산진구 부전동', '연자제보쌈 부산광역시 동래구 온천동', '커피텍 로스터스 부산광역시 동구 범일동', '진가네손칼국수 부산광역시 사하구 당리동', '케이닭 부산광역시 연제구 거제동', '유명산곰장어 부산광역시 부산진구 부전동', '브레이크아웃 이스케이프 부산광역시 해운대구 중동', '교촌치킨 덕천2호점 부산광역시 북구 덕천동', '동해대게 부산광역시 강서구 신호동', '포도청 부산광역시 부산진구 부전동', '주차장산곰장어 1호점 부산광역시 부산진구 부전동', '남해멧돌순두부 부산광역시 연제구 거제동', '할매김밥 

In [22]:
columns = ['title','subway','reviewer','review','rank']
df_restaurants = pd.DataFrame(columns=columns)

columns2 = ['title','address']
df_unsearched = pd.DataFrame(columns=columns2)

In [30]:
# 검색

restaurants = ['서울깍두기 부산광역시 부산진구 부전동', '대복집 부산광역시 연제구 연산동']
# restaurants = ['서울깍두기 부산광역시 부산진구 부전동']
for i in range(len(restaurants)) :
    
#     driver = webdriver.Chrome('C:/Users/SSAFY/Desktop/selenium/chromedriver.exe',chrome_options=chrome_options)
    driver = webdriver.Chrome('C:/Users/SSAFY/Desktop/selenium/chromedriver.exe')
    driver.implicitly_wait(5)
    url = 'https://map.naver.com/v5/search'
    driver.get(url)
    driver.implicitly_wait(5)
    search_box = driver.find_element_by_css_selector('div.input_box>input.input_search')
    driver.implicitly_wait(5)
    search_box.send_keys(restaurants_list[i])
    driver.implicitly_wait(5)
    search_box.send_keys(Keys.ENTER)
    driver.implicitly_wait(5)
    

    searchiframe = driver.find_element_by_id("searchIframe")
    driver.switch_to.frame(searchiframe)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser') 
    
    try :
        try:
            driver.switch_to.default_content()

            entryIframe = driver.find_element_by_id("entryIframe")
            driver.switch_to.frame(entryIframe)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_detail_wrapper = soup.select_one('div.place_detail_wrapper')
        except:
            element = driver.find_element_by_css_selector('#_pcmap_list_scroll_container > ul > li:nth-child(1) > div._3ZU00 > a:nth-child(1)')
            driver.execute_script("arguments[0].click();", element)
            driver.implicitly_wait(10)

            driver.switch_to.default_content()

            entryIframe = driver.find_element_by_id("entryIframe")
            driver.switch_to.frame(entryIframe)

            html = driver.page_source # 클릭 후 페이지가져오기
            soup = BeautifulSoup(html, 'html.parser')
            place_detail_wrapper = soup.select_one('div.place_detail_wrapper')
    except:
        df_unsearched = df_unsearched.append({
            'title': restaurants_title[i],
            'address' : restaurants_address[i]
        },ignore_index=True)

    try :


        # 가게이름
        try :
            title = place_detail_wrapper.select('span._3XamX')[0].text
            print(title)
        except :
            title = ''
        
            
        # 지하철
        try :
            subway_tag = place_detail_wrapper.select('div._2P6sT')[0]
            while 1 :
                if subway_tag.find('span') :
                    subway_tag.find('span').decompose() # span태그 빼기
                else :
                    break
            subway = subway_tag.text
            print(subway)
        except :
            subway =''
        
        # 리뷰
        driver.implicitly_wait(5)
        find_review_table = soup.select_one('div._2MDmw')
        driver.implicitly_wait(5)
        # print(driver.find_element_by_css_selector('div.place_detail_wrapper > div.place_fixed_maintab > div > div > div > div > a:nth-child(1) > span')
        for i in range(len(find_review_table)) :
            path = 'div.place_detail_wrapper > div.place_fixed_maintab > div > div > div > div > a:nth-child({0}) > span'.format(i+1)
            if driver.find_element_by_css_selector(path).text == '리뷰':
                driver.find_element_by_css_selector(path).click()
                break

        flag = 1
        click_n = 0
        driver.implicitly_wait(5)
        while flag :
            if click_n >= 10 :
                break
            try :
                driver.find_element_by_css_selector('div._2kAri').click()
                time.sleep(0.5)
                click_n += 1
            except :
                flag = 0
        html = driver.page_source # 내리고 나서의 페이지를 싹 가져옴
        soup = BeautifulSoup(html, 'html.parser') 
        reviews_ul = soup.select_one('ul._1jVSG')
        reviews_li = reviews_ul.select('li')

        for i in range(len(reviews_li)):
#             print(len(reviews_li))
            
            #리뷰어
            try :
                reviewer = reviews_li[i].select('div._16RxQ')[0].text
#                 print(reviewer)
            except :
                reviewer = ''
            #리뷰
            try :
                review = reviews_li[i].select('span.WoYOw')[0].text
#                 print(review)
            except :
                review = ''
            # 랭크
            try :
                rank = reviews_li[i].select('span.Sv1wj')[0].text[2:]
#                 print(rank)
            except :
                rank = ''
#                 print(rank)
            df_restaurants = df_restaurants.append({
                'title':title,
                'subway':subway,
                'reviewer' : reviewer,
                'review':review,
                'rank':rank
            },ignore_index=True)
            
    except :
        pass

    driver.switch_to.default_content()
    print(len(df_restaurants))
    
    driver.quit()
#     time.sleep(1)


서울깍두기
서면역 12번 출구에서127m
74
대복집
시청역 5번 출구에서454m
107


In [31]:
for i in range(len(restaurants)) :
    
#     driver = webdriver.Chrome('C:/Users/SSAFY/Desktop/selenium/chromedriver.exe',chrome_options=chrome_options)
    driver = webdriver.Chrome('C:/Users/SSAFY/Desktop/selenium/chromedriver.exe')
    driver.implicitly_wait(5)
    url = 'https://map.naver.com/v5/search'
    driver.get(url)
    driver.implicitly_wait(5)
    search_box = driver.find_element_by_css_selector('div.input_box>input.input_search')
    driver.implicitly_wait(5)
    search_box.send_keys(restaurants_list[i])
    driver.implicitly_wait(5)
    search_box.send_keys(Keys.ENTER)
    driver.implicitly_wait(5)
    

    searchiframe = driver.find_element_by_id("searchIframe")
    driver.switch_to.frame(searchiframe)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser') 
    
    try :
        driver.switch_to.default_content()

        entryIframe = driver.find_element_by_id("entryIframe")
        driver.switch_to.frame(entryIframe)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        place_detail_wrapper = soup.select_one('div.place_detail_wrapper')
    except :
        element = driver.find_element_by_css_selector('#_pcmap_list_scroll_container > ul > li:nth-child(1) > div._3ZU00 > a:nth-child(1)')
        driver.execute_script("arguments[0].click();", element)
        driver.implicitly_wait(10)

        driver.switch_to.default_content()

        entryIframe = driver.find_element_by_id("entryIframe")
        driver.switch_to.frame(entryIframe)

        html = driver.page_source # 클릭 후 페이지가져오기
        soup = BeautifulSoup(html, 'html.parser')
        place_detail_wrapper = soup.select_one('div.place_detail_wrapper')